In [ ]:
import pandas
events = pandas.read_csv('data/events.csv', sep=';', skiprows=1)
persons = pandas.read_csv('data/persons.csv', sep=';', skiprows=1)
documents = pandas.read_csv('data/documents.csv', sep=';', skiprows=1)

gedcom = GedcomReader().read_gedcom('data/exmp.ged')
print(len(gedcom.persons))
for family in gedcom.families:
    print(family)

In [8]:
import upload
archive = upload.create_package(r'D:\site', 'data')
print(archive)
# scp data.zip kzhagorina@ssh.pythonanywhere.com:~/genotree/upload/data.zip

D:\site\data.zip


In [ ]:
def split_binary_file(filename, chunk_size=50*1024*1024):
    """split file into chunks of size"""
    
    f = open(filename, 'rb')
    data = f.read()
    f.close()

    count_chunks = int(len(data)/chunk_size + bool(len(data)%chunk_size))
    for i in range(0, count_chunks):
        chunk_filename = filename+".{0:000}".format(i+1)
        f = open(chunk_filename, 'wb+')
        f.write(data[i*chunk_size: (i+1)*chunk_size])
        f.close()

In [ ]:
split_binary_file(r'D:\site\data.zip')

In [9]:
import upload
archive = r'D:\site\data.zip'
upload.load_package(archive, 
                    r'C:\Users\Ksenia\Documents\IPython Notebooks\genotree\src\static\tree',
                    r'C:\Users\Ksenia\Documents\IPython Notebooks\genotree\data\tree')

undefined: [PAGE http://ok.ru/profile/121643311] in {}


In [ ]:
from gedcom import Gedcom, GedcomReader, GedcomWriter
gedcom = GedcomReader().read_gedcom('D:/site/tree.ged')

In [ ]:
gedcom = GedcomReader().read_gedcom('data/exmp.ged')